Implementation of the self-calibration pipeline on an ETH3D SLAM dataset.  
Given only a sequence of RGB images from a single moving camera, it:
1. Loads an ETH3D monocular sequence, applies frame stride / frame cap, and resizes copies of frames for feature extraction.
2. Detects local features (SIFT/ORB) and matches them between frames.
3. Finds consistent frame pairs and estimates Fundamental matrices (F) using mutual matches, RANSAC, and parallax checks.
4. Estimates the camera intrinsics K (focal length and principal point) from epipolar geometry using an SVD-based score and a 1D search over f (with fallback to a heuristic prior if too few stable F are available).
5. Initializes two seed camera poses from a high-parallax pair, checks cheirality, and triangulates initial 3D points.
6. Grows the model across frames using PnP on 2D–3D correspondences induced by existing 3D tracks and inter-frame matches, then triangulates new points from matched keypoints between registered views.
7. Filters outliers by reprojection error and positive depth, and builds a cleaned, capped observation set for optimization.
8. Refines intrinsics with a fast intrinsics-only BA.
9. Runs a small “full” BA on subsets (intrinsics + selected poses + selected 3D points) with a robust loss.
10. Outputs the final intrinsics K and the reconstructed camera poses and 3D points (which can then be saved or visualized).


In [74]:
!pip -q install opencv-python-headless==4.10.0.84 numpy scipy

import os, sys, glob, json, shutil, zipfile, pathlib
import numpy as np
import cv2
from matplotlib import pyplot as plt
from dataclasses import dataclass
from typing import List, Tuple, Dict
import re
import math
import random

np.random.seed(0)
random.seed(0)
cv2.setRNGSeed(0)

In [75]:
# Download a monocular sequence from the ETH3D SLAM dataset
import urllib.request, os, zipfile

root = "/content/eth3d"
os.makedirs(root, exist_ok=True)
name = "einstein_1"   # dataset's name
# name = "einstein_global_light_changes_2"
mono_url = f"https://www.eth3d.net/data/slam/datasets/{name}_mono.zip"
dst_zip  = f"{root}/{name}_mono.zip"
dst_dir  = f"{root}/training"

os.makedirs(dst_dir, exist_ok=True)
print("Downloading:", mono_url)
urllib.request.urlretrieve(mono_url, dst_zip)

with zipfile.ZipFile(dst_zip, 'r') as z:
    z.extractall(dst_dir)
os.remove(dst_zip)

# Locate folder with RGB images
rgb_dir = glob.glob(f"{dst_dir}/{name}/**/rgb", recursive=True)[0]
print("Images dir:", rgb_dir, "num imgs:", len(glob.glob(rgb_dir+'/*.png')))


Downloading: https://www.eth3d.net/data/slam/datasets/einstein_1_mono.zip
Images dir: /content/eth3d/training/einstein_1/rgb num imgs: 487


In [76]:
# Load all PNG images from the RGB directory of ETH3D format
img_paths = sorted(glob.glob(os.path.join(rgb_dir, "*.png")))
assert len(img_paths) > 0, f"No PNGs in {rgb_dir}"
images = [cv2.imread(p, cv2.IMREAD_COLOR) for p in img_paths]
assert all(im is not None for im in images), "Some images failed to load"

# Extract image dimensions
H, W = images[0].shape[:2]
print(f"Loaded {len(images)} frames, HxW={H}x{W}")


Loaded 487 frames, HxW=458x739


In [77]:
# Define control parameters for speed/robustness
STRIDE     = 4      # Downsample frame sequence by this stride
MAX_FRAMES = 150    # Limit total number of frames
TARGET_W   = 960   # Rescale width (used for feature extraction)
FEATURE    = "sift" # Feature type: "sift" | "orb"

# List of F-matrices that passed geometric checks (for self-calibration)
F_candidates = []


# Apply frame stride and limit number of frames
images = images[::STRIDE][:MAX_FRAMES]
H, W = images[0].shape[:2]
cx, cy = W * 0.5, H * 0.5 # Principal point is assumed to be the image center
print(f"Using {len(images)} frames after stride, HxW={H}x{W}")

# Initialize intrinsic matrix K
f0 = 1.2 * max(H, W)
fx = float(f0)
fy = float(f0)
K = np.array([[fx, 0.0, cx],
              [0.0, fy,  cy],
              [0.0, 0.0, 1.0]], dtype=np.float64)

# Store intrinsic priors (used later for regularization in Bundle Adjustment)
K_prior = (fx, fy, cx, cy)


# Data container for storing keypoints and descriptors per frame
@dataclass
class FrameData:
    kps: List[cv2.KeyPoint] # List of keypoints detected in the image
    desc: np.ndarray        # Corresponding descriptor matrix
    pts_px: np.ndarray      # Pixel coordinates (N x 2) at original resolution

# Create a feature extractor
def create_feature(name="sift"):
    name = (name or "sift").lower()
    if name == "sift":
        try:
            return cv2.SIFT_create(nfeatures=3000)
        except Exception:
            pass
    return cv2.ORB_create(nfeatures=3000, scaleFactor=1.2, nlevels=10, edgeThreshold=15, fastThreshold=7)

# Apply CLAHE (Contrast Limited Adaptive Histogram Equalization) to enhance image contrast
def preprocess_gray(g):
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    return clahe.apply(g)

# Convert an image to grayscale
def to_gray(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) if img.ndim==3 else img

def match_descriptors(d1, d2, ratio=0.80, min_matches=50):
    """
    Perform two-way descriptor matching:
    - Forward: KNN + Lowe's ratio test.
    - Reverse: 1-NN verification to ensure mutual correspondence.
    Returns a list of matches that satisfy both filters.
    """
    # Sanity checks to ensure both descriptor sets are non-empty
    if d1 is None or d2 is None:
        return []
    if len(d1) == 0 or len(d2) == 0:
        return []

    # Choose matcher type based on descriptor data type
    is_bin = (d1.dtype == np.uint8 and d2.dtype == np.uint8)
    if is_bin:
        matcher_fwd = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
        matcher_rev = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)
        d1_use, d2_use = d1, d2
        d2_use_rev, d1_use_rev = d2, d1
    else:
        d1_use = np.asarray(d1, dtype=np.float32)
        d2_use = np.asarray(d2, dtype=np.float32)
        d2_use_rev = d2_use
        d1_use_rev = d1_use
        index_params = dict(algorithm=1, trees=5)
        search_params = dict(checks=64)
        matcher_fwd = cv2.FlannBasedMatcher(index_params, search_params)
        matcher_rev = cv2.FlannBasedMatcher(index_params, search_params)

    # Perform KNN matching (k=2) for Lowe's ratio filtering
    knn = matcher_fwd.knnMatch(d1_use, d2_use, k=2)
    ratio_pass = []
    for pair in knn:
        if len(pair) < 2:
            continue
        m, n = pair
        if m.distance < ratio * n.distance:
            ratio_pass.append(m)

    if not ratio_pass:
        return []

    # Perform reverse 1-NN matching to check mutual correspondence
    knn_rev = matcher_rev.knnMatch(d2_use_rev, d1_use_rev, k=1)
    rev_map = {}
    for pair in knn_rev:
        if not pair:
            continue
        r = pair[0]
        rev_map[r.queryIdx] = r.trainIdx

    # Mutual match check: only retain matches where forward and reverse agree
    mutual = []
    seen = set()  # avoid duplicates
    for m in ratio_pass:
        if rev_map.get(m.trainIdx, -1) == m.queryIdx:
            key = (m.queryIdx, m.trainIdx)
            if key not in seen:
                seen.add(key)
                mutual.append(m)

    # If mutual matches are too few, skip further processing
    if len(mutual) < min_matches:
        return []

    return mutual

# Robust estimation of the Fundamental matrix
def ransac_F(pts1, pts2):
    F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC,
                                     ransacReprojThreshold=3.0, confidence=0.995, maxIters=4000)
    # Retry with more iterations if initial estimation fails
    if F is None or F.shape!=(3,3):
        F, mask = cv2.findFundamentalMat(pts1, pts2, cv2.FM_RANSAC,
                                         ransacReprojThreshold=3.0, confidence=0.995, maxIters=8000)
    mask = (mask.ravel().astype(bool) if mask is not None else np.zeros(len(pts1), bool))
    return F, mask


Using 122 frames after stride, HxW=458x739


In [78]:
feat = create_feature(FEATURE)

# Initialize the list to store extracted feature information for each frame
frames: List[FrameData] = []

for img in images:
    # Compute the scaling factor to resize the image to the target working width
    scale = min(1.0, TARGET_W / float(img.shape[1]))
    img_small = cv2.resize(img, (int(img.shape[1]*scale), int(img.shape[0]*scale)),
                           interpolation=cv2.INTER_AREA) if scale<1.0 else img
    gray = preprocess_gray(to_gray(img_small))

    # Detect keypoints and compute descriptors using the selected feature detector
    kps_small, desc = feat.detectAndCompute(gray, None)

    # Map keypoints back to original scale so the rest of geometry uses full-res pixels
    kps, pts_px = [], []
    inv = 1.0 / scale

    if scale == 1.0:
        # No scaling was applied
        kps = kps_small
        pts_px = [kp.pt for kp in kps_small]
    else:
        for kp in kps_small:
            x = kp.pt[0] * inv
            y = kp.pt[1] * inv

            # Scale the keypoint size appropriately and preserve its original attributes
            size   = max(1.0, kp.size * inv)
            angle  = float(kp.angle) if kp.angle is not None else -1.0
            resp   = float(kp.response)
            octave = int(kp.octave)
            clsid  = int(kp.class_id)

            new_kp = cv2.KeyPoint(float(x), float(y), float(size), float(angle), resp, octave, clsid)

            # Append the new keypoint and its position to the lists
            kps.append(new_kp)
            pts_px.append([x, y])

    pts_px = np.array(pts_px, dtype=np.float32)
    frames.append(FrameData(kps=kps, desc=desc, pts_px=pts_px))


print("Features extracted for", len(frames), "frames")


Features extracted for 122 frames


In [79]:
# List to store high-quality image pairs with sufficient parallax and valid epipolar geometry
good_pairs = []

# Helper function to skip visually similar consecutive frames
def frames_too_similar(i, j, thr=1.5):
    g1 = to_gray(images[i]).astype(np.float32)
    g2 = to_gray(images[j]).astype(np.float32)
    return float(np.mean(np.abs(g1 - g2))) < thr

# Main loop to iterate over the frame sequence and identify geometrically valid pairs
i = 0
while i < len(frames)-1:
    tried = False

    # Try pairing current frame with the next or the one after
    for j in (i+1, i+2 if i+2 < len(frames) else i+1):
        if j >= len(frames) or frames_too_similar(i,j):
            continue
        f1, f2 = frames[i], frames[j]

        # Match descriptors with Lowe's ratio test + mutual check
        matches = match_descriptors(f1.desc, f2.desc, ratio=0.85)
        if len(matches) < 60:
            continue

        # Extract matched keypoint coordinates
        pts1 = np.array([f1.kps[m.queryIdx].pt for m in matches], np.float32)
        pts2 = np.array([f2.kps[m.trainIdx].pt for m in matches], np.float32)
        F, mask = ransac_F(pts1, pts2)
        inl = int(mask.sum())

        print(f"[debug] {i}-{j} matches = {len(matches)}")
        if F is not None:
            print(f"[debug]     inliers = {inl}, cond(F) = {np.linalg.cond(F):.1e}")

        # Check quality of fundamental matrix and number of inliers
            if F is not None and inl >= 30 and np.linalg.cond(F) < 1e20:
              good = [matches[k] for k in range(len(matches)) if mask[k]]

              pts1_u = cv2.undistortPoints(
                  pts1[mask].reshape(-1, 1, 2), K, None
              ).reshape(-1, 2)
              pts2_u = cv2.undistortPoints(
                  pts2[mask].reshape(-1, 1, 2), K, None
              ).reshape(-1, 2)

              E_tmp = K.T @ F @ K

              _, Rtmp, ttmp, _ = cv2.recoverPose(
                  E_tmp,
                  pts1_u.reshape(-1, 1, 2),
                  pts2_u.reshape(-1, 1, 2),
              )

              b1 = np.column_stack([pts1_u, np.ones(len(pts1_u))])
              b2 = (Rtmp @ np.column_stack([pts2_u, np.ones(len(pts2_u))]).T).T
              b1 /= np.linalg.norm(b1, axis=1, keepdims=True)
              b2 /= np.linalg.norm(b2, axis=1, keepdims=True)

              ang = np.degrees(np.arccos(np.clip(np.sum(b1 * b2, axis=1), -1, 1)))
              if np.median(ang) < 1.5:
                  continue

              good_pairs.append((i, j, good, F))

              F_candidates.append(F.copy())

              i = j
              tried = True
              break


    if not tried:
        print(f"[warn] Skipping weak pair at {i}-{i+1}")
        i += 1
    if len(good_pairs) >= 60:
        print(f"[info] Enough good pairs: {len(good_pairs)}. Stop.")
        break

assert len(good_pairs) > 0, "No good pairs found. Try larger STRIDE or different sequence."
print("Good pairs:", len(good_pairs))


[debug] 0-1 matches = 112
[debug]     inliers = 97, cond(F) = 1.8e+17
[debug] 1-2 matches = 115
[debug]     inliers = 102, cond(F) = 6.4e+17
[debug] 2-3 matches = 143
[debug]     inliers = 130, cond(F) = 1.7e+17
[debug] 3-4 matches = 170
[debug]     inliers = 157, cond(F) = 3.0e+17
[debug] 3-5 matches = 143
[debug]     inliers = 129, cond(F) = 1.4e+18
[debug] 5-6 matches = 197
[debug]     inliers = 186, cond(F) = 8.9e+16
[debug] 6-7 matches = 214
[debug]     inliers = 203, cond(F) = 2.0e+17
[debug] 7-8 matches = 252
[debug]     inliers = 241, cond(F) = 5.6e+18
[debug] 7-9 matches = 227
[debug]     inliers = 201, cond(F) = 2.5e+17
[debug] 9-10 matches = 211
[debug]     inliers = 184, cond(F) = 8.0e+16
[debug] 10-11 matches = 234
[debug]     inliers = 217, cond(F) = 9.3e+17
[debug] 11-12 matches = 285
[debug]     inliers = 264, cond(F) = 1.2e+20
[debug] 11-13 matches = 211
[debug]     inliers = 194, cond(F) = 3.4e+18
[debug] 13-14 matches = 249
[debug]     inliers = 232, cond(F) = 1.2e+1

In [80]:
# Estimate initial focal length from Fundamental matrix via grid search and SVD analysis
def estimate_focal_from_F_list(F_list, W, H, verbose=True):
    if len(F_list) < 3:
        # Too few matrices for stable estimation
        f_heur = 1.2 * max(W, H)
        if verbose:
            print(f"[Focal] Not enough F-matrices ({len(F_list)}), fallback f≈{f_heur:.1f}")
        return float(f_heur)

    maxWH = float(max(W, H))

    def score_for_f(f):
        # Build K(f)
        K = np.array(
            [
                [f, 0.0, W * 0.5],
                [0.0, f, H * 0.5],
                [0.0, 0.0, 1.0],
            ],
            dtype=np.float64,
        )

        total = 0.0
        count = 0
        for F in F_list:
            E = K.T @ F @ K
            U, S, Vt = np.linalg.svd(E)
            s1, s2, s3 = S

            # Skip degenerate cases
            if s1 <= 0.0 or s2 <= 0.0:
                continue

            num = (s1 - s2) ** 2 + s3 ** 2
            den = s1 * s2
            total += num / den
            count += 1

        if count == 0:
            return np.inf
        return total / count

    f_min = 0.8 * maxWH
    f_max = 1.8 * maxWH
    n_samples = 50

    f_grid = np.linspace(f_min, f_max, n_samples)
    scores = []
    for f in f_grid:
        s = score_for_f(f)
        scores.append(s)

    scores = np.asarray(scores, dtype=np.float64)
    best_idx = int(np.argmin(scores))
    f_best = float(f_grid[best_idx])

    if verbose:
        print(f"[Focal] coarse search on [{f_min:.1f}, {f_max:.1f}] px, "
              f"best≈{f_best:.2f}, score={scores[best_idx]:.3e}")

    # Local refinement around the best f
    width = (f_max - f_min) / n_samples * 4.0
    f_ref_min = max(f_min, f_best - width)
    f_ref_max = min(f_max, f_best + width)

    n_ref = 30
    f_ref_grid = np.linspace(f_ref_min, f_ref_max, n_ref)
    ref_scores = []
    for f in f_ref_grid:
        s = score_for_f(f)
        ref_scores.append(s)

    ref_scores = np.asarray(ref_scores, dtype=np.float64)
    best_idx2 = int(np.argmin(ref_scores))
    f_opt = float(f_ref_grid[best_idx2])

    if verbose:
        print(
            f"[Focal] refine search on [{f_ref_min:.1f}, {f_ref_max:.1f}] px, "
            f"f≈{f_opt:.2f}, score={ref_scores[best_idx2]:.3e}"
        )

    return float(f_opt)


F_list = [F for (_, _, _, F) in good_pairs]

print(f"[SelfCal] Using {len(F_list)} F-matrices for focal estimation")

if len(F_list) < 3:
    f_init = 1.2 * max(W, H)
    print(f"[SelfCal] Not enough F-matrices ({len(F_list)}), fallback f≈{f_init:.1f}")
else:
    f_init = estimate_focal_from_F_list(F_list, W, H, verbose=True)

cx = W * 0.5
cy = H * 0.5

fx = float(f_init)
fy = float(f_init)
K  = np.array(
    [
        [fx, 0.0, cx],
        [0.0, fy, cy],
        [0.0, 0.0, 1.0],
    ],
    dtype=np.float64,
)

K_prior = (fx, fy, cx, cy)
print(f"[Init intrinsics] f≈{fx:.1f} px, cx={cx:.1f}, cy={cy:.1f}")

[SelfCal] Using 60 F-matrices for focal estimation
[Focal] coarse search on [591.2, 1330.2] px, best≈591.20, score=3.183e-03
[Focal] refine search on [591.2, 650.3] px, f≈591.20, score=3.183e-03
[Init intrinsics] f≈591.2 px, cx=369.5, cy=229.0


In [ ]:
# Triangulates 3D points from corresponding normalized image coordinates
def triangulate_points(P0, P1, x0, x1):
    x0_h = np.vstack([x0.T, np.ones((1,x0.shape[0]))])
    x1_h = np.vstack([x1.T, np.ones((1,x1.shape[0]))])
    X_h = cv2.triangulatePoints(P0, P1, x0_h[:2,:], x1_h[:2,:])
    X = (X_h[:3,:]/X_h[3,:]).T.copy()
    return X

# Computes the median parallax angle between rays originating from two camera positions to each triangulated 3D point
def parallax_angle(R, t, pts3d):
    """
    Estimates median parallax angle between rays from two cameras to 3D points.
    R, t: rotation and translation from camera 1 to 2
    pts3d: Nx3 triangulated 3D points (in world frame of camera 1)
    Returns angle in degrees.
    """
    rays1 = pts3d / np.linalg.norm(pts3d, axis=1, keepdims=True)
    rays2 = ((R @ pts3d.T) + t.reshape(3, 1)).T
    rays2 = rays2 / np.linalg.norm(rays2, axis=1, keepdims=True)

    cos_angles = np.clip(np.sum(rays1 * rays2, axis=1), -1.0, 1.0)
    angles_rad = np.arccos(cos_angles)
    return np.median(angles_rad) * 180.0 / np.pi  # degrees

# Projects 3D points X into 2D image coordinates using camera intrinsics K and extrinsic parameters given by rvec and tvec
def project_points(X, rvec, tvec, K):
    img_pts, _ = cv2.projectPoints(X.astype(np.float32),
                                   rvec.reshape(3,1).astype(np.float32),
                                   tvec.reshape(3,1).astype(np.float32),
                                   K.astype(np.float32), None)
    return img_pts.reshape(-1,2)


# Fast vectorized pinhole projection
def pinhole_project_batch(f: float, cx: float, cy: float,
                          R: np.ndarray, t: np.ndarray,
                          X: np.ndarray) -> np.ndarray:
    X = np.asarray(X, np.float64)
    R = np.asarray(R, np.float64).reshape(3, 3)
    t = np.asarray(t, np.float64).reshape(3,)
    Xc = (R @ X.T).T + t
    z  = np.clip(Xc[:, 2:3], 1e-9, None)
    u  = Xc[:, :2] / z
    u  = u * float(f) + np.array([cx, cy], dtype=np.float64)
    return u


# Select first seed pair (i0, j0) and estimate relative pose
def choose_seed_pair(good_pairs, K, frames, min_parallax_deg=1.0):
    """
    Select a seed-a pair of frames with sufficient parallax.
    """
    for (i0, j0, matches0, F01) in good_pairs:
        pts1 = np.array([frames[i0].kps[m.queryIdx].pt for m in matches0], np.float32)
        pts2 = np.array([frames[j0].kps[m.trainIdx].pt for m in matches0], np.float32)

        E = K.T @ F01 @ K
        pts1_u = cv2.undistortPoints(pts1.reshape(-1, 1, 2), K, None).reshape(-1, 2)
        pts2_u = cv2.undistortPoints(pts2.reshape(-1, 1, 2), K, None).reshape(-1, 2)

        _, R01, t01, inl01 = cv2.recoverPose(
            E,
            pts1_u.reshape(-1, 1, 2),
            pts2_u.reshape(-1, 1, 2),
        )
        inl01 = inl01.ravel().astype(bool)

        if inl01.sum() < 50:
            print(f"[seed] pair ({i0},{j0}) rejected: inliers={inl01.sum()}")
            continue

        x1_seed = pts1_u[inl01]
        x2_seed = pts2_u[inl01]
        b1 = np.column_stack([x1_seed, np.ones(len(x1_seed))])
        b2 = (R01 @ np.column_stack([x2_seed, np.ones(len(x2_seed))]).T).T
        b1 /= np.linalg.norm(b1, axis=1, keepdims=True)
        b2 /= np.linalg.norm(b2, axis=1, keepdims=True)
        ang = np.degrees(np.arccos(np.clip(np.sum(b1 * b2, axis=1), -1, 1)))
        ang_med = np.median(ang)

        if ang_med < min_parallax_deg:
            print(f"[seed] pair ({i0},{j0}) rejected: parallax={ang_med:.2f} deg")
            continue

        print(f"[seed] using pair ({i0},{j0}), parallax={ang_med:.2f} deg, inliers={inl01.sum()}")
        return i0, j0, matches0, F01, R01, t01, inl01, pts1, pts2, pts1_u, pts2_u

    i0, j0, matches0, F01 = good_pairs[0]
    print(f"[seed] fallback to first pair ({i0},{j0}) without parallax check")
    pts1 = np.array([frames[i0].kps[m.queryIdx].pt for m in matches0], np.float32)
    pts2 = np.array([frames[j0].kps[m.trainIdx].pt for m in matches0], np.float32)
    E = K.T @ F01 @ K
    pts1_u = cv2.undistortPoints(pts1.reshape(-1, 1, 2), K, None).reshape(-1, 2)
    pts2_u = cv2.undistortPoints(pts2.reshape(-1, 1, 2), K, None).reshape(-1, 2)
    _, R01, t01, inl01 = cv2.recoverPose(
        E,
        pts1_u.reshape(-1, 1, 2),
        pts2_u.reshape(-1, 1, 2),
    )
    inl01 = inl01.ravel().astype(bool)
    return i0, j0, matches0, F01, R01, t01, inl01, pts1, pts2, pts1_u, pts2_u


# Choose seed pair with decent parallax
i0, j0, matches0, F01, R01, t01, inl01, pts1, pts2, pts1_u, pts2_u = \
    choose_seed_pair(good_pairs, K, frames, min_parallax_deg=1.0)

# Camera poses, 3D points and observations initialization
poses: Dict[int, Tuple[np.ndarray,np.ndarray]] = {}
points3d: Dict[int, np.ndarray] = {}
observations: Dict[int, List[Tuple[int, np.ndarray]]] = {}

from collections import defaultdict

obs_by_frame: Dict[int, List[Tuple[np.ndarray, int]]] = defaultdict(list)
kp2pid_by_frame: Dict[int, Dict[int, int]] = defaultdict(dict)

def add_observation(pid: int, frame_id: int, m_px: np.ndarray):
    observations.setdefault(pid, []).append((frame_id, m_px))
    obs_by_frame[frame_id].append((m_px, pid))


poses[i0] = (np.zeros(3), np.zeros(3))
rvec1, _ = cv2.Rodrigues(R01)
poses[j0] = (rvec1.ravel(), t01.ravel())

# Triangulate initial 3D points from first good pair
x1n = pts1_u[inl01]
x2n = pts2_u[inl01]
P0 = np.hstack([np.eye(3), np.zeros((3,1))])
P1 = np.hstack([R01, t01])
X01 = triangulate_points(P0, P1, x1n, x2n)
angle = parallax_angle(R01, t01, X01)


# Filter and validate initial triangulated 3D points between first two frames
SEED_REPROJ_THRESH = 5.0

# Filter and validate initial triangulated 3D points between first two frames
kept = 0
# indices of inlier matches in the original matches0 list
inlier_idx = np.where(inl01)[0]

for j, X in enumerate(X01):
    idx = inlier_idx[j]
    m0  = matches0[idx]
    kp_i = m0.queryIdx
    kp_j = m0.trainIdx

    z0 = (np.eye(3) @ X + np.zeros(3))[2]
    z1 = (R01 @ X + t01.ravel())[2]
    if z0 <= 0 or z1 <= 0:
        continue

    # Reprojection error sanity check for both views
    pr0 = project_points(X[None,:], np.zeros(3), np.zeros(3), K)[0]
    pr1 = project_points(X[None,:], cv2.Rodrigues(R01)[0].ravel(), t01.ravel(), K)[0]

    if np.linalg.norm(pr0 - pts1[inl01][j]) < SEED_REPROJ_THRESH and \
       np.linalg.norm(pr1 - pts2[inl01][j]) < SEED_REPROJ_THRESH:

        pid = len(points3d)
        points3d[pid] = X

        add_observation(pid, i0, pts1[inl01][j])
        add_observation(pid, j0, pts2[inl01][j])

        kp2pid_by_frame[i0][kp_i] = pid
        kp2pid_by_frame[j0][kp_j] = pid

        kept += 1

print(f"[seed] 3D points kept after reproj filter: {kept}")

if kept < 30:
    print(f"[seed] Fallback: using cheirality-only filter (was kept={kept})")
    points3d.clear()
    observations.clear()
    obs_by_frame.clear()
    kp2pid_by_frame.clear()
    kept = 0

    for j, X in enumerate(X01):
        idx = inlier_idx[j]
        m0  = matches0[idx]
        kp_i = m0.queryIdx
        kp_j = m0.trainIdx

        z0 = (np.eye(3) @ X + np.zeros(3))[2]
        z1 = (R01 @ X + t01.ravel())[2]
        if z0 <= 0 or z1 <= 0:
            continue

        pid = len(points3d)
        points3d[pid] = X

        add_observation(pid, i0, pts1[inl01][j])
        add_observation(pid, j0, pts2[inl01][j])

        kp2pid_by_frame[i0][kp_i] = pid
        kp2pid_by_frame[j0][kp_j] = pid

        kept += 1

print(f"[seed] total seed 3D points kept: {kept}")

if kept < 20:
    print("[warn] Very few seed 3D points, reconstruction may be unstable")

MIN_PNP_POINTS = 12  # minimum number of 2D-3D correspondences for a reliable PnP
# Begin incremental reconstruction using remaining good image pairs
for (ia, ib, matches, F_ab) in good_pairs[1:]:
    if ia not in poses:
        continue

    f_cur, f_nxt = frames[ia], frames[ib]
    pts2d, pts3d_list = [], []

    map_a = kp2pid_by_frame.get(ia, {})
    for m in matches:
        pid = map_a.get(m.queryIdx, None)
        if pid is None:
            continue
        X = points3d.get(pid, None)
        if X is None:
            continue

        pts3d_list.append(X)
        pts2d.append(f_nxt.kps[m.trainIdx].pt)

    if len(pts3d_list) < MIN_PNP_POINTS:
        # Not enough constraints for a stable PnP -> skip this pair
        continue

    pts3d = np.asarray(pts3d_list, np.float32)
    pts2d = np.asarray(pts2d, np.float32)

    success, rvec, tvec, inl = cv2.solvePnPRansac(
        pts3d, pts2d,
        K, None,
        iterationsCount=2000,
        reprojectionError=2.5,
        confidence=0.999
    )

    if (not success) or (inl is None) or (len(inl) < MIN_PNP_POINTS):
        # PnP failed or too few inliers
        continue

    # Reject views with insufficient parallax for reliable triangulation
    pts_i = np.array([f_cur.kps[m.queryIdx].pt for m in matches], np.float32)
    pts_j = np.array([f_nxt.kps[m.trainIdx].pt for m in matches], np.float32)

    pts_i_u = cv2.undistortPoints(pts_i.reshape(-1, 1, 2), K, None).reshape(-1, 2)
    pts_j_u = cv2.undistortPoints(pts_j.reshape(-1, 1, 2), K, None).reshape(-1, 2)

    R_est, _ = cv2.Rodrigues(rvec)
    b1 = np.column_stack([pts_i_u, np.ones(len(pts_i_u))])
    b2 = (R_est @ np.column_stack([pts_j_u, np.ones(len(pts_j_u))]).T).T
    b1 /= np.linalg.norm(b1, axis=1, keepdims=True)
    b2 /= np.linalg.norm(b2, axis=1, keepdims=True)
    ang = np.degrees(np.arccos(np.clip(np.sum(b1 * b2, axis=1), -1, 1)))
    if np.median(ang) < 3.0:
        continue

    # Pose assignment
    poses.setdefault(ia, (np.zeros(3), np.zeros(3)))
    poses[ib] = (rvec.ravel(), tvec.ravel())

    # Triangulate new points
    R_i, _ = cv2.Rodrigues(poses[ia][0]); t_i = poses[ia][1].reshape(3, 1)
    R_j, _ = cv2.Rodrigues(poses[ib][0]); t_j = poses[ib][1].reshape(3, 1)
    P_i = np.hstack([R_i, t_i])
    P_j = np.hstack([R_j, t_j])

    pts_i = np.array([f_cur.kps[m.queryIdx].pt for m in matches], np.float32)
    pts_j = np.array([f_nxt.kps[m.trainIdx].pt for m in matches], np.float32)

    pts_i_u = cv2.undistortPoints(pts_i.reshape(-1, 1, 2), K, None).reshape(-1, 2)
    pts_j_u = cv2.undistortPoints(pts_j.reshape(-1, 1, 2), K, None).reshape(-1, 2)

    X_ij = triangulate_points(P_i, P_j, pts_i_u, pts_j_u)


    # Validate and store newly triangulated points
    for k, Xk in enumerate(X_ij):
        z_i = (R_i @ Xk + t_i.ravel())[2]
        z_j = (R_j @ Xk + t_j.ravel())[2]
        if z_i <= 0 or z_j <= 0:
            continue

        pr_i = project_points(Xk[None, :], poses[ia][0], poses[ia][1], K)[0]
        pr_j = project_points(Xk[None, :], poses[ib][0], poses[ib][1], K)[0]

        if (np.linalg.norm(pr_i - pts_i[k]) >= 2.5) or (np.linalg.norm(pr_j - pts_j[k]) >= 2.5):
            continue

        # Link triangulated point to existing 3D point if keypoint in ia is already mapped
        m = matches[k]
        kp_i = m.queryIdx
        kp_j = m.trainIdx

        existing_pid = kp2pid_by_frame[ia].get(kp_i, None)

        if existing_pid is not None:
            # Add new observation for ib
            add_observation(existing_pid, ib, pts_j[k])
            kp2pid_by_frame[ib][kp_j] = existing_pid
        else:
            # Register a new 3D point and map both keypoints to it
            pid = len(points3d)
            points3d[pid] = Xk

            add_observation(pid, ia, pts_i[k])
            add_observation(pid, ib, pts_j[k])

            kp2pid_by_frame[ia][kp_i] = pid
            kp2pid_by_frame[ib][kp_j] = pid


[seed] using pair (0,1), parallax=7.79 deg, inliers=63
[seed] 3D points kept after reproj filter: 63
[seed] total seed 3D points kept: 63


In [82]:
# Accumulate all reprojection errors for each 3D point across all its observations
all_errs = []
track_err = {}

# Iterate over all 3D points and their associated 2D observations
for pid, obs in observations.items():
    errs = []
    for (fi, m_px) in obs:
        if fi not in poses:
            continue
        rvec, tvec = poses[fi]

        # Project the 3D point into the image using the current estimated camera pose
        pr = project_points(points3d[pid][None,:], rvec, tvec, K)[0]
        errs.append(float(np.linalg.norm(pr - m_px)))
    if errs:
        track_err[pid] = errs
        all_errs.extend(errs)

# Detect outliers using robust statistics
if all_errs:
    # Compute first and third quartile of reprojection errors
    q1, q3 = np.percentile(all_errs, [25, 75])
    iqr = max(q3 - q1, 1e-6)
    thr = q3 + 2.0*iqr

    # Remove outlier observations for each 3D point
    for pid, errs in list(track_err.items()):
        keep_obs = []
        for (fi, m_px), e in zip(observations[pid], errs):
            if e < thr:
                keep_obs.append((fi, m_px))
        if len(keep_obs) >= 2:
            observations[pid] = keep_obs
        else:
            observations.pop(pid, None)
            points3d.pop(pid, None)


In [ ]:
try:
    from scipy.optimize import least_squares
    SCIPY_OK = True
except Exception:
    SCIPY_OK = False

def ba_refine_intrinsics_only(
    observations, poses, points3d, f0, cx0, cy0, W, H,
    max_obs_total=12000, verbose=True,
):
    if not SCIPY_OK:
        if verbose: print("[BA intr] SciPy NA")
        return f0, cx0, cy0

    # Use pre-cleaned obs if available; else fallback to raw
    global clean_obs
    triplets = []
    seen = 0
    for (pid, fi, m_px) in clean_obs if 'clean_obs' in globals() else []:
        if pid in points3d and fi in poses:
            triplets.append((pid, fi, m_px))
            seen += 1
            if seen >= max_obs_total:
                break
    if not triplets:
        # fallback: raw, very small cap
        for pid, obs in observations.items():
            if pid not in points3d: continue
            for (fi, m_px) in obs:
                if fi not in poses: continue
                triplets.append((pid, fi, np.asarray(m_px, np.float64)))
                if len(triplets) >= min(4000, max_obs_total):
                    break
            if len(triplets) >= min(4000, max_obs_total):
                break

    if len(triplets) < 200:
        if verbose: print(f"[BA intr] too few obs: {len(triplets)}")
        return f0, cx0, cy0

    # Group by camera for batch projection
    by_cam = {}
    for pid, fi, m in triplets:
        by_cam.setdefault(int(fi), {"X":[], "m":[]}).update({})
        by_cam[int(fi)]["X"].append(points3d[pid])
        by_cam[int(fi)]["m"].append(m)

    cam_ids = sorted(by_cam.keys())
    R_list, t_list, X_batches, m_batches = [], [], [], []
    for fid in cam_ids:
        rvec, tvec = poses[fid]
        R, _ = cv2.Rodrigues(np.asarray(rvec, np.float64).reshape(3,1))
        R_list.append(R)
        t_list.append(np.asarray(tvec, np.float64).reshape(3,))
        X_batches.append(np.asarray(by_cam[fid]["X"], np.float64))
        m_batches.append(np.asarray(by_cam[fid]["m"], np.float64))

    f0 = float(f0); cx0 = float(cx0); cy0 = float(cy0)
    maxWH = float(max(W,H))

    def residuals(theta):
        f, cx, cy = theta
        res = []
        for R, t, Xb, Mb in zip(R_list, t_list, X_batches, m_batches):
            proj = pinhole_project_batch(f, cx, cy, R, t, Xb)
            res.append((proj - Mb).ravel())
        res = np.concatenate(res, axis=0)

        # Weak priors
        lam_f = 5e-6
        lam_c = 5e-5
        pri  = np.array([
            lam_f * (f  - f0) / maxWH,
            lam_c * (cx - cx0) / W,
            lam_c * (cy - cy0) / H
        ], dtype=np.float64)
        return np.concatenate([res, pri], axis=0)

    theta0 = np.array([f0, cx0, cy0], np.float64)
    lb = np.array([0.55*maxWH, 0.25*W, 0.15*H], np.float64)
    ub = np.array([2.20*maxWH, 0.75*W, 0.85*H], np.float64)

    result = least_squares(
        residuals, x0=theta0, bounds=(lb,ub),
        loss="soft_l1", f_scale=2.0,
        max_nfev=12, verbose=2 if verbose else 0
    )
    f_opt, cx_opt, cy_opt = result.x
    if verbose:
        print(f"[BA intr] f {f0:.2f}->{f_opt:.2f}, cx {cx0:.2f}->{cx_opt:.2f}, cy {cy0:.2f}->{cy_opt:.2f}")
    return float(f_opt), float(cx_opt), float(cy_opt)



def ba_refine_f_cx_cy_full(
    observations,
    poses,
    points3d,
    f0,
    cx0,
    cy0,
    W,
    H,
    max_frames_ba: int = 15,
    max_points_ba: int = 800,
    max_obs_total: int = 10000,
    verbose: bool = True,
):
    """
    Mini full bundle-adjustment:

      - Optimizes intrinsics (f, cx, cy)
      - Optimizes poses (rvec, tvec) for a subset of cameras
      - Optimizes 3D points for a subset of tracks

    All residuals are reprojection errors in pixels computed via cv2.projectPoints.
    """

    if not SCIPY_OK:
        print("[BA] SciPy not available, skipping BA")
        return float(f0), float(cx0), float(cy0)

    if not poses or not points3d or not observations:
        print("[BA] Empty poses/points/observations, skipping BA")
        return float(f0), float(cx0), float(cy0)

    total_frames = len(poses)
    total_points = len(points3d)
    total_obs_raw = sum(len(v) for v in observations.values())
    print(f"[BA] total frames={total_frames}, total points={total_points}, total raw obs={total_obs_raw}")

    # Select subset of frames for BA
    frame_ids = sorted(int(fi) for fi in poses.keys())
    if not frame_ids:
        print("[BA] No frame ids, skipping BA")
        return float(f0), float(cx0), float(cy0)

    if len(frame_ids) > max_frames_ba:
        step = max(1, len(frame_ids) // max_frames_ba)
        frame_ids = frame_ids[::step]

    frame_set = set(frame_ids)
    base_fid = frame_ids[0]       # base camera: fixed pose
    opt_frame_ids = [fid for fid in frame_ids if fid != base_fid]
    cam_index = {fid: idx for idx, fid in enumerate(opt_frame_ids)}
    n_cams_opt = len(opt_frame_ids)

    print(f"[BA] frames selected for BA: {len(frame_ids)} (base={base_fid}, opt={n_cams_opt})")

    # Build subset of points and observations
    pt_ids      = []
    X0_list     = []
    obs_pt_idx  = []
    obs_cam_idx = []
    obs_meas    = []

    pts_used = 0
    obs_used = 0

    for pid, X in points3d.items():
        obs_full = observations.get(pid, [])
        if not obs_full:
            continue

        # Take only obs from selected frames
        obs_f = [(int(fi), m) for (fi, m) in obs_full if int(fi) in frame_set]
        if len(obs_f) < 2:
            continue

        pt_idx = len(pt_ids)
        pt_ids.append(pid)
        X0_list.append(np.asarray(X, dtype=np.float64).reshape(3,))

        for (fi, m_px) in obs_f:
            if fi not in poses:
                continue

            # Determine camera index for this frame
            if fi == base_fid:
                ci = -1
            else:
                ci = cam_index.get(fi, -1)
                if ci == -1:
                    # This frame is not in the subset of cameras we optimize
                    continue
            obs_pt_idx.append(pt_idx)
            obs_cam_idx.append(ci)
            obs_meas.append(np.asarray(m_px, dtype=np.float64).reshape(2,))

            obs_used += 1
            if obs_used >= max_obs_total:
                break

        pts_used += 1
        if pts_used >= max_points_ba or obs_used >= max_obs_total:
            break

    if obs_used < 60 or pts_used < 30 or len(obs_meas) < 40:
        print(f"[BA] Not enough obs/points for BA (points_used={pts_used}, obs_used={obs_used}), skip")
        return float(f0), float(cx0), float(cy0)

    X0 = np.vstack(X0_list).astype(np.float64)
    obs_meas    = np.vstack(obs_meas).astype(np.float64)
    obs_pt_idx  = np.asarray(obs_pt_idx,  dtype=np.int32)
    obs_cam_idx = np.asarray(obs_cam_idx, dtype=np.int32)

    used_frames = sorted({base_fid} | {opt_frame_ids[ci] for ci in obs_cam_idx if ci >= 0})
    print(f"[BA] frames actually used in obs: {len(used_frames)}")

    # Initial extrinsics for opt cameras
    rvecs0 = np.zeros((n_cams_opt, 3), dtype=np.float64)
    tvecs0 = np.zeros((n_cams_opt, 3), dtype=np.float64)
    for fid in opt_frame_ids:
        if fid not in poses:
            continue
        rvec, tvec = poses[fid]
        rvecs0[cam_index[fid]] = np.asarray(rvec, dtype=np.float64).reshape(3,)
        tvecs0[cam_index[fid]] = np.asarray(tvec, dtype=np.float64).reshape(3,)

    f0  = float(f0)
    cx0 = float(cx0)
    cy0 = float(cy0)
    maxWH = float(max(W, H))

    print(f"[BA] initial intrinsics: f={f0:.2f}, cx={cx0:.2f}, cy={cy0:.2f}")

    def pack_params(f, cx, cy, rvecs, tvecs, X):
        return np.concatenate(
            [
                np.array([f, cx, cy], dtype=np.float64),
                rvecs.ravel(),
                tvecs.ravel(),
                X.ravel(),
            ]
        )

    def unpack_params(theta):
        theta = np.asarray(theta, dtype=np.float64).ravel()
        f, cx, cy = theta[0], theta[1], theta[2]
        off = 3

        rvecs = rvecs0.copy()
        tvecs = tvecs0.copy()

        if n_cams_opt > 0:
            rvecs = theta[off : off + 3 * n_cams_opt].reshape(n_cams_opt, 3)
            off += 3 * n_cams_opt
            tvecs = theta[off : off + 3 * n_cams_opt].reshape(n_cams_opt, 3)
            off += 3 * n_cams_opt

        X = theta[off:].reshape(X0.shape[0], 3)
        return float(f), float(cx), float(cy), rvecs, tvecs, X

    theta0 = pack_params(f0, cx0, cy0, rvecs0, tvecs0, X0)

    lb = np.full_like(theta0, -np.inf, dtype=np.float64)
    ub = np.full_like(theta0, +np.inf, dtype=np.float64)

    f_lo = 0.5 * maxWH
    f_hi = 2.5 * maxWH
    cx_lo, cx_hi = 0.1 * W, 0.9 * W
    cy_lo, cy_hi = 0.1 * H, 0.9 * H

    lb[0], ub[0] = f_lo, f_hi
    lb[1], ub[1] = cx_lo, cx_hi
    lb[2], ub[2] = cy_lo, cy_hi

    def compute_rms(f, cx, cy, rvecs, tvecs, X):
        K_loc = np.array(
            [
                [f, 0.0, cx],
                [0.0, f, cy],
                [0.0, 0.0, 1.0],
            ],
            dtype=np.float32,
        )
        err2 = []
        for k in range(obs_meas.shape[0]):
            j = int(obs_pt_idx[k])
            ci = int(obs_cam_idx[k])

            if ci >= 0:
                r = rvecs[ci]
                t = tvecs[ci]
            else:
                r = np.zeros(3, dtype=np.float64)
                t = np.zeros(3, dtype=np.float64)

            Xj = X[j].reshape(1, 3).astype(np.float32)
            r_ = r.reshape(3, 1).astype(np.float32)
            t_ = t.reshape(3, 1).astype(np.float32)
            proj, _ = cv2.projectPoints(Xj, r_, t_, K_loc, None)
            proj = proj.reshape(2,)
            diff = proj - obs_meas[k]
            err2.append(float(diff.dot(diff)))
        return math.sqrt(sum(err2) / len(err2))

    rms0 = compute_rms(f0, cx0, cy0, rvecs0, tvecs0, X0)
    print(f"[BA] initial RMS reprojection error: {rms0:.4f} px (subset)")



    obs_idx_by_cam = {}
    for k in range(obs_meas.shape[0]):
        ci = int(obs_cam_idx[k])
        obs_idx_by_cam.setdefault(ci, []).append(k)
    for ci in list(obs_idx_by_cam.keys()):
        obs_idx_by_cam[ci] = np.asarray(obs_idx_by_cam[ci], np.int32)


    # Residuals for least_squares
    def residuals(theta):
        f, cx, cy, rvecs, tvecs, X = unpack_params(theta)

        # Precompute rotation matrices for optimized cameras
        Rs = np.zeros((n_cams_opt, 3, 3), np.float64)
        for i in range(n_cams_opt):
            R_i, _ = cv2.Rodrigues(rvecs[i].reshape(3,1))
            Rs[i] = R_i

        res_chunks = []

        idx = obs_idx_by_cam.get(-1, None)
        if idx is not None and idx.size:
            j_idx = obs_pt_idx[idx]
            Xb    = X[j_idx]
            Rb    = np.eye(3, dtype=np.float64)
            tb    = np.zeros(3, dtype=np.float64)
            proj  = pinhole_project_batch(f, cx, cy, Rb, tb, Xb)
            res_chunks.append((proj - obs_meas[idx]).ravel())

        # Optimized cameras
        for ci in range(n_cams_opt):
            idx = obs_idx_by_cam.get(ci, None)
            if idx is None or not idx.size:
                continue
            j_idx = obs_pt_idx[idx]
            Xb    = X[j_idx]
            proj  = pinhole_project_batch(f, cx, cy, Rs[ci], tvecs[ci], Xb)
            res_chunks.append((proj - obs_meas[idx]).ravel())

        res = np.concatenate(res_chunks, axis=0) if res_chunks else np.zeros(0, np.float64)

        # Soft priors
        lam_f    = 1e-5
        lam_c    = 1e-4
        lam_pose = 5e-5
        lam_X    = 5e-6

        pri = [
            lam_f * (f  - f0)  / maxWH,
            lam_c * (cx - cx0) / W,
            lam_c * (cy - cy0) / H,
        ]
        pri.extend((lam_pose * (rvecs.ravel() - rvecs0.ravel())).tolist())
        pri.extend((lam_pose * (tvecs.ravel() - tvecs0.ravel())).tolist())
        pri.extend((lam_X    * (X.ravel()     - X0.ravel())).tolist())

        return np.concatenate([res, np.asarray(pri, np.float64)], axis=0)


    # Run optimizer
    try:
        print("[BA] starting full mini-BA optimization...")
        result = least_squares(
            residuals,
            x0=theta0,
            bounds=(lb, ub),
            loss="soft_l1",
            f_scale=2.0,
            max_nfev=15,
            verbose=2 if verbose else 0,
        )
        f_opt, cx_opt, cy_opt, rvecs_opt, tvecs_opt, X_opt = unpack_params(result.x)
        rms1 = compute_rms(f_opt, cx_opt, cy_opt, rvecs_opt, tvecs_opt, X_opt)
        print(f"[BA] optimization finished, status={result.status}, nfev={result.nfev}")
        print(
            f"[BA] intrinsics refined: "
            f"f:  {f0:.2f} -> {f_opt:.2f}, "
            f"cx: {cx0:.2f} -> {cx_opt:.2f}, "
            f"cy: {cy0:.2f} -> {cy_opt:.2f}"
        )
        print(f"[BA] final   RMS reprojection error: {rms1:.4f} px (subset)")

        # Optionally update poses and points back into global dicts
        for fid in opt_frame_ids:
            if fid not in poses:
                continue
            idx = cam_index[fid]
            poses[fid] = (rvecs_opt[idx].copy(), tvecs_opt[idx].copy())

        for local_idx, pid in enumerate(pt_ids):
            points3d[pid] = X_opt[local_idx].copy()

    except Exception as e:
        print(f"[BA] least_squares failed with exception: {e}")
        f_opt, cx_opt, cy_opt = f0, cx0, cy0

    return float(f_opt), float(cx_opt), float(cy_opt)



# Pre-BA outlier culling and subsampling
def _proj_err_px(K, rvec, tvec, X, m):
    R, _ = cv2.Rodrigues(np.asarray(rvec, np.float64).reshape(3,1))
    Xc = (R @ X.T + np.asarray(tvec, np.float64).reshape(3,1)).T
    z  = Xc[:, 2:3]
    good = z[:, 0] > 1e-6
    u = Xc[:, :2] / np.clip(z, 1e-6, None)
    u = u * K[0,0] + np.array([K[0,2], K[1,2]], dtype=np.float64)
    err = np.linalg.norm(u - m, axis=1)
    return err, good

def build_clean_subset(observations, poses, points3d, K,
                       per_frame_cap=800,
                       err_thresh=3.0):
    """Return filtered observations: list of (pid, fi, m_px) only for inliers."""
    kept = []
    for pid, obs in observations.items():
        if pid not in points3d:
            continue
        X = np.asarray(points3d[pid], np.float64).reshape(1,3)
        # Group obs by frame
        byf = {}
        for (fi, m) in obs:
            if fi in poses:
                byf.setdefault(int(fi), []).append(np.asarray(m, np.float64).reshape(2,))
        for fi, ms in byf.items():
            rvec, tvec = poses[fi]
            M = np.stack(ms, axis=0)
            Xrep = np.repeat(X, len(ms), axis=0)
            err, good = _proj_err_px(K, rvec, tvec, Xrep, M)
            idx = np.where((good) & (err < err_thresh))[0]
            if idx.size:
                if idx.size > per_frame_cap:
                    idx = idx[:per_frame_cap]
                for j in idx:
                    kept.append((pid, fi, M[j]))
    return kept

clean_obs = build_clean_subset(observations, poses, points3d, K, per_frame_cap=600, err_thresh=3.0)
print(f"[Pre-BA] kept obs after culling: {len(clean_obs)}")


f_ba, cx_ba, cy_ba = ba_refine_intrinsics_only(
    observations=observations,
    poses=poses,
    points3d=points3d,
    f0=f_init,
    cx0=cx,
    cy0=cy,
    W=W,
    H=H,
    max_obs_total=20000,
    verbose=True,
)

fx_tmp = float(f_ba)
fy_tmp = float(f_ba)
cx_tmp = float(cx_ba)
cy_tmp = float(cy_ba)

K_tmp = np.array(
    [
        [fx_tmp, 0.0,   cx_tmp],
        [0.0,    fy_tmp, cy_tmp],
        [0.0,    0.0,   1.0],
    ],
    dtype=np.float64,
)

print(f"[After intr-BA] f≈{fx_tmp:.1f} px, cx={cx_tmp:.1f}, cy={cy_tmp:.1f}")

f_refined, cx_ref, cy_ref = ba_refine_f_cx_cy_full(
    observations=observations,
    poses=poses,
    points3d=points3d,
    f0=f_ba,
    cx0=cx_ba,
    cy0=cy_ba,
    W=W,
    H=H,
    max_frames_ba=20,
    max_points_ba=1200,
    max_obs_total=15000,
    verbose=True,
)


# Post-processing: safety checks and final K
f_floor = 0.6 * max(W, H)
if (not np.isfinite(f_refined)) or (f_refined < f_floor):
    print(f"[BA] refined f={f_refined:.2f} is too small, fallback to >= {f_floor:.2f}")
    f_refined = max(f_ba, f_floor)

cx_ref = float(np.clip(cx_ref, 0.3 * W, 0.7 * W))
cy_ref = float(np.clip(cy_ref, 0.3 * H, 0.7 * H))

K_ref = np.array(
    [
        [f_refined, 0.0,      cx_ref],
        [0.0,       f_refined, cy_ref],
        [0.0,       0.0,      1.0],
    ],
    dtype=np.float64,
)

K  = K_ref.copy()
fx = float(K[0, 0]); fy = float(K[1, 1])
cx = float(K[0, 2]); cy = float(K[1, 2])

print(f"[Refined intrinsics] f≈{fx:.1f} px, cx={cx:.1f}, cy={cy:.1f}")



[Pre-BA] kept obs after culling: 2919
   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.5339e+03                                    7.97e+03    
       1              2         1.5335e+03      3.61e-01       6.44e-02       3.78e-01    
       2              3         1.5335e+03      1.03e-09       3.33e-06       3.34e-05    
Both `ftol` and `xtol` termination conditions are satisfied.
Function evaluations 3, initial cost 1.5339e+03, final cost 1.5335e+03, first-order optimality 3.34e-05.
[BA intr] f 591.20->591.26, cx 369.50->369.49, cy 229.00->228.99
[After intr-BA] f≈591.3 px, cx=369.5, cy=229.0
[BA] total frames=16, total points=1047, total raw obs=3065
[BA] frames selected for BA: 16 (base=0, opt=15)
[BA] frames actually used in obs: 16
[BA] initial intrinsics: f=591.26, cx=369.49, cy=228.99
[BA] initial RMS reprojection error: 1.3598 px (subset)
[BA] starting full mini-BA optimization...
   Iteration     To

In [84]:
out_dir = "/content/out_sfm"
os.makedirs(out_dir, exist_ok=True)
np.save(os.path.join(out_dir, "K.npy"), K)


K_est = np.load("/content/out_sfm/K.npy")
fx_e, fy_e, cx_e, cy_e = K_est[0,0], K_est[1,1], K_est[0,2], K_est[1,2]

seq_dir = str(pathlib.Path(rgb_dir).parent)
calib_path = os.path.join(seq_dir, "calibration.txt")
print("calibration.txt:", calib_path, os.path.exists(calib_path))

fx_gt=fy_gt=cx_gt=cy_gt=None
if os.path.exists(calib_path):
    with open(calib_path, "r") as f:
        txt = f.read()
    nums = re.findall(r"[-+]?\d*\.\d+|\d+", txt)
    vals = list(map(float, nums))
    for i in range(len(vals)-3):
        fx_gt, fy_gt, cx_gt, cy_gt = vals[i:i+4]
        if 50 < fx_gt < 20000 and 50 < fy_gt < 20000:
            break

    if fx_gt is not None:
        rel_fx = abs(fx_e - fx_gt)/fx_gt
        rel_fy = abs(fy_e - fy_gt)/fy_gt
        dx = abs(cx_e - cx_gt); dy = abs(cy_e - cy_gt)

        print(f"[GT compare] fx: est={fx_e:.1f} gt={fx_gt:.1f}  rel.err={100*rel_fx:.2f}%")
        print(f"[GT compare] fy: est={fy_e:.1f} gt={fy_gt:.1f}  rel.err={100*rel_fy:.2f}%")
        print(f"[GT compare] cx: est={cx_e:.1f} gt={cx_gt:.1f}  |Δ|={dx:.2f}px")
        print(f"[GT compare] cy: est={cy_e:.1f} gt={cy_gt:.1f}  |Δ|={dy:.2f}px")
    else:
        print("Ground-truth intrinsics not parsed; check file format.")
else:
    print("No ground truth file found for this sequence (test set or different layout).")


calibration.txt: /content/eth3d/training/einstein_1/calibration.txt True
[GT compare] fx: est=602.2 gt=726.3  rel.err=17.08%
[GT compare] fy: est=602.2 gt=726.3  rel.err=17.08%
[GT compare] cx: est=368.6 gt=354.6  |Δ|=13.94px
[GT compare] cy: est=228.1 gt=186.5  |Δ|=41.63px


In [85]:
import numpy as np
import cv2
from collections import defaultdict
import math

def reprojection_metrics_both(K, poses, points3d, observations, dist_opt=None):
    """
    Compute two families of metrics:
      1) 'Chessboard-style' score (as in many Zhang tutorials): L2 / N per frame, then averaged.
      2) Proper reprojection errors:
           - per-frame RMSE
           - global RMSE (recommended)
           - also global MAE and median for reference

    Returns
    -------
    summary : dict
        {
          "mean_chess_like": float,     # mean over frames of (L2 / N)
          "mean_rmse_frames": float,    # mean over frames of per-frame RMSE
          "global_rmse": float,         # sqrt( sum ||e||^2 / total_points )
          "global_mae": float,          # mean ||e||
          "global_median": float        # median ||e||
        }
    per_frame : dict[int -> dict]
        Each entry: {"chess_like": ..., "rmse": ..., "mae": ..., "N": int}
    """

    per_frame_X, per_frame_m = defaultdict(list), defaultdict(list)
    for pid, obs_list in observations.items():
        X = points3d.get(pid)
        if X is None:
            continue
        X = np.asarray(X, np.float32).reshape(3,)
        for (fi, m_px) in obs_list:
            if fi in poses:
                per_frame_X[int(fi)].append(X)
                per_frame_m[int(fi)].append(np.asarray(m_px, np.float32).reshape(2,))

    K32   = np.asarray(K, np.float32)
    dist32 = None if dist_opt is None else np.asarray(dist_opt, np.float32)

    # Per-frame metrics
    per_frame = {}
    chess_vals = []
    rmse_vals  = []

    all_errs = []   # accumulate per-point errors over all frames for global metrics

    for fi in sorted(per_frame_X.keys()):
        if fi not in poses:
            continue

        X3d = np.asarray(per_frame_X[fi], np.float32)
        m2d = np.asarray(per_frame_m[fi], np.float32)
        if X3d.size == 0:
            continue

        rvec, tvec = poses[fi]
        r32 = np.asarray(rvec, np.float32).reshape(3,1)
        t32 = np.asarray(tvec, np.float32).reshape(3,1)

        proj, _ = cv2.projectPoints(X3d, r32, t32, K32, dist32)  # (N,1,2)
        proj = proj.reshape(-1, 2)

        diff = proj - m2d
        N = len(diff)

        # L2 / N
        chess_like = cv2.norm(diff.reshape(-1,1,2), cv2.NORM_L2) / max(N, 1)

        # 2) Proper errors
        e = np.linalg.norm(diff, axis=1)          # per-point L2 in pixels
        mae_i  = float(np.mean(e))                # mean absolute error (per frame, optional)
        rmse_i = float(math.sqrt(np.mean(e**2)))  # RMSE per frame

        per_frame[fi] = {
            "chess_like": float(chess_like),
            "rmse": rmse_i,
            "mae": mae_i,
            "N": int(N),
        }
        chess_vals.append(float(chess_like))
        rmse_vals.append(rmse_i)
        all_errs.append(e)

    if all_errs:
        all_errs = np.concatenate(all_errs, axis=0)
        global_rmse   = float(math.sqrt(np.mean(all_errs**2)))
        global_mae    = float(np.mean(all_errs))
        global_median = float(np.median(all_errs))
    else:
        global_rmse = global_mae = global_median = float("nan")

    mean_chess_like  = float(np.mean(chess_vals)) if chess_vals else float("nan")
    mean_rmse_frames = float(np.mean(rmse_vals))  if rmse_vals  else float("nan")

    summary = {
        "mean_chess_like":  mean_chess_like,   # matches the chess-style formula
        "mean_rmse_frames": mean_rmse_frames,  # average of per-frame RMSE
        "global_rmse":      global_rmse,       # preferred single number for reports
        "global_mae":       global_mae,
        "global_median":    global_median,
    }
    return summary, per_frame


summary, per_frame = reprojection_metrics_both(K, poses, points3d, observations, dist_opt=None)
print("Chessboard-style mean (L2 divided by N):", summary["mean_chess_like"])
print("Mean per-frame RMSE:",  summary["mean_rmse_frames"])
print("Global RMSE (px):   ",  summary["global_rmse"])
worst = sorted(per_frame.items(), key=lambda kv: kv[1]["rmse"], reverse=True)[:5]
print("Worst 5 frames by RMSE:", *worst)


Chessboard-style mean (L2 divided by N): 0.06734798229153924
Mean per-frame RMSE: 0.8894977616969626
Global RMSE (px):    0.8969364065488314
Worst 5 frames by RMSE: (11, {'chess_like': 0.07418023220359976, 'rmse': 1.0620992690205697, 'mae': 0.8538821339607239, 'N': 205}) (9, {'chess_like': 0.0713900885437266, 'rmse': 1.056476855322834, 'mae': 0.868816077709198, 'N': 219}) (15, {'chess_like': 0.0686077427854186, 'rmse': 1.0106551423116137, 'mae': 0.7742008566856384, 'N': 217}) (5, {'chess_like': 0.06911791510961861, 'rmse': 0.9676507743139702, 'mae': 0.7760979533195496, 'N': 196}) (13, {'chess_like': 0.08055879028005135, 'rmse': 0.9599687919411792, 'mae': 0.717512309551239, 'N': 142})
